<a href="https://colab.research.google.com/github/chasubeen/TalesRunnner/blob/subeen/koT5_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Google Drive 연결
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 저장 경로 설정
data_path = "/content/drive/MyDrive/7th-project/data"
model_path = "/content/drive/MyDrive/7th-project/model/koT5"

# **1. 데이터 준비**


## **1-1. 데이터 불러오기**
- 전처리 된 csv 파일 불러오기
- 해당 csv 파일에는 다음과 같은 정보들을 포함하고 있음

| Field | Description |
| --- | --- |
| id |* 이미지 → 캡션 정보와 annotation 정보를 결합 |
| imgpath | * `동화 삽화 생성 데이터 > 라벨링 데이터` 내의 jpg 파일 |
| caption | * BLIP을 통과시켜 얻어진 캡션|
| srcText | * LM의 output text|
| name | * 객체의 일반명사 (필수)|
| i_action | * 객체의 행동, 상태 (필수) |
| classification | * 의사소통/자연탐구/사회관계/예술경험/신체운동_건강 (필수) |
| character | * 캐릭터 (선택) |
| setting | * 셋팅 (선택) |
| action | * 행동 (선택) |
| feeling | * 감정 (선택) |
| causalRelationship | * 인과 (선택) |
| outcomeResolution | * 결과 (선택) |
| prediction | * 예측 (선택) |

In [ ]:
import pandas as pd

In [ ]:
## 전처리된 데이터(csv 파일) 불러오기
train_data = pd.read_csv(f"{data_path}/dataset_train.csv") # 학습
val_data = pd.read_csv(f"{data_path}/dataset_val.csv") # 검증

print(f"Train Data: {len(train_data)} samples")
print(f"Validation Data: {len(val_data)} samples")

Train Data: 40001 samples
Validation Data: 5000 samples


In [ ]:
train_data.head()

,id,img_path,caption,srcText,name,i_action,classification,character,setting,action,feeling,causalRelationship,outcomeResolution,prediction
0,03_05T_03S_9791157023899_42712,/content/data/TL_05T_신체운동_건강_03S_초...,"드레스와 모자를 쓴 세 명의 여성, 한 명은 모자를 쓰고 있습니다.","“플로렌스, 왜 리처드와 결혼을 하지 않겠다는 거니?” 어머니가 물었어요. 언니 파...",성인 여자,입에 한 손을 대고 놀란다,신체운동_건강,"플로렌스,어머니,파세노프,간호사",NaN,"물었어요,돌보며,되겠어요",NaN,NaN,NaN,NaN
1,03_05T_03S_9791157023998_39246,/content/data/TL_05T_신체운동_건강_03S_초...,책상에 앉아 있는 가족의 만화,"그래, 일본으로 가자. 호랑이 굴로 들어가 보는 거야.’ 일본에서 벗어나려면 일본을...",남성,펜을 잡고 있다.,신체운동_건강,"방정환,일본,어린이",동양 대학,"가자,들어가 보는,알아야,결심했어요,건너간,들어가,공부했어요,누려야",NaN,NaN,NaN,NaN
2,03_05T_03S_9791157023813_60463,/content/data/TL_05T_신체운동_건강_03S_초...,갈색 옷과 모자를 쓴 중국인,백 번을 죽고 또 죽어도 고려 임금을 향한 신의를 저버리지 않겠다는 뜻이었어요. 정...,성인 남자,모자를 썼다,신체운동_건강,"고려 임금,정몽주,이방원,자객",선죽교,보낸,NaN,NaN,비록 아쉽게 생을 마쳤지만 정몽주는 지금까지 충절을 지킨 가장 위대한 위인으로 기억...,NaN
3,03_05T_03S_9791157023981_34117,/content/data/TL_05T_신체운동_건강_03S_초...,한 여성이 남자와 개와 함께 텐트 앞에 앉아 있다,밀림을 떠나 케임브리지 대학에서 공부하고 왔는데도\n침팬지들은 제인을 잊지 않았던 ...,성인 여성,바나나를 들고 있다.,신체운동_건강,"침팬지들,제인,바나나,휴고","케임브리지 대학,밀림","떠나,공부하고,왔는데도,들어와,먹으며,놀려 댔어요,장난을 걸어요",NaN,NaN,NaN,NaN
4,03_05T_03S_9791157024247_33953,/content/data/TL_05T_신체운동_건강_03S_초...,여자가 컴퓨터가 있는 책상에 앉아 있다,"그런데 그날 저녁, 엄마가 뜻밖의 이야기를 하셨어요. ""승찬아, 이번 주 일요일은 ...",남자 아이,앉아 있다,신체운동_건강,"승찬이,엄마,할아버지,사촌 형,누나들","그날 저녁,일요일,토요일,하룻밤,저녁","하셨어요,하룻밤 자고,돌아올 거야,올 거래",NaN,NaN,NaN,NaN


In [ ]:
val_data.head()

,id,img_path,caption,srcText,name,i_action,classification,character,setting,action,feeling,causalRelationship,outcomeResolution,prediction
0,03_02T_03S_9788998212643_54879,/content/data/VL_02T_자연탐구_03S_초등_고하...,횡단보도로 길을 건너는 사람들의 집단,“출발합니다! 손잡이 꼭 잡으세요.” 삐이익. 다음 정류장에 내릴 손님이 벨을 눌렀...,신호등,초록색이다,자연탐구,"손잡이,정류장,손님,벨,차돌,신호등",NaN,"출발합니다,잡으세요,내릴,눌렀어요,멈췄다 가자,지키는,지켜야 해요",NaN,NaN,NaN,NaN
1,03_02T_03S_9788967760144_35692,/content/data/VL_02T_자연탐구_03S_초등_고하...,보트 위에 있는 남자와 여자의 그림,두 번째로 간 곳은 바다였다. “이번 모험은 고기잡이배에서 생활하는 것이다.” 마음...,고기잡이배,바다에 떠 있다,자연탐구,"모험,나는,그물,물고기,파도","바다,고기잡이배,하루 종일","출발했다,생활하는,간,태운,먹지 못하고,끌어 올려야",NaN,NaN,NaN,멀고 먼 바다로 출발했다.
2,03_02T_03S_9788998212636_5162,/content/data/VL_02T_자연탐구_03S_초등_고하...,들판에 있는 동물 무리의 그림,제주도에 여행 온 친구들은 힝힝 씨가 들려주는 제주도 이야기를 아주 좋아해요. “제...,해녀,바다에 있다,자연탐구,"친구,힝힝 씨,해녀,섬",제주도,들려주는,좋아해요,NaN,NaN,NaN
3,03_02T_03S_9791128211331_32163,/content/data/VL_02T_자연탐구_03S_초등_고하...,침대에서 자는 아이의 그림,쿠르쿠르는 에코스 덕분에 아주 오랜만에 단잠을 잘 수 있었어. 하지만 단 하루뿐이었...,난쟁이,침대에 누워있다,자연탐구,"쿠르쿠르,에코스",다음 날,"잘 수 있었어,들리기 시작한 거야",끔찍한,NaN,NaN,NaN
4,03_02T_03S_9791128211195_59242,/content/data/VL_02T_자연탐구_03S_초등_고하...,만화 캐릭터가 있는 빨간색과 주황색 배경,“이 원반은 사람의 핏속에 있는 적혈구랍니다. 적혈구는 사람의 몸속 구석구석으로 산...,적혈구,붉은색 원반 모양을 하고 흘러가고 있어요.,자연탐구,"원반,사람,적혈구,산소",핏속,"나르지요,여행할 거예요",NaN,NaN,NaN,NaN


## **1-2. text 전처리**





### **input text**
- special token을 활용하여 각 정보를 구분
  - 필수 필드
    - 결측치 확인(결측치 허용 x)
  - 선택 필드
    - 다중 아이템의 경우 (,)로 구분 후 각 항목에 토큰 부여
    - 결측치가 있는 경우 `<empty>` 토큰 부여
- caption과 보조적 정보를 포함하여 구성
  - 행별 고유 시드 설정: 각 행의 id(또는 index)를 seed로 설정   
  → 동일한 데이터를 사용하면 언제나 같은 결과가 나오도록 보장
  - 랜덤 순서 보장: 각 행마다 token의 순서를 랜덤하게 설정  
  → 모델이 token 간 순서나 위치에 지나치게 의존하지 않고, 각 요소의 의미와 역할을 학습하도록 유도

In [ ]:
import random

In [ ]:
### Input Text 생성 함수

def generate_input_text(row):
    ## 고유 seed 설정
    random.seed(hash(row["id"]))


    ## 필수 필드
    # 존재성 보장(assertion)
    assert pd.notna(row['caption']) and row['caption'].strip(), "Error: 'caption' 필드는 비워둘 수 없습니다."
    assert pd.notna(row['name']) and row['name'].strip(), "Error: 'name' 필드는 비워둘 수 없습니다."
    assert pd.notna(row['i_action']) and row['i_action'].strip(), "Error: 'i_action' 필드는 비워둘 수 없습니다."
    assert pd.notna(row['classification']) and row['classification'].strip(), "Error: 'classification' 필드는 비워둘 수 없습니다."

    required_fields = [
        f"<caption> {row['caption']}",
        f"<name> {row['name']}",
        f"<i_action> {row['i_action']}",
        f"<classification> {row['classification']}"
    ]


    ## 선택 필드
    optional_fields = {
        "character": "<character>",
        "setting": "<setting>",
        "action": "<action>",
        "feeling": "<feeling>",
        "causalRelationship": "<causalRelationship>",
        "outcomeResolution": "<outcomeResolution>",
        "prediction": "<prediction>"
    }

    optional_tokens = []
    for field, token in optional_fields.items():
        if pd.notna(row[field]) and row[field].strip():
            # 여러 항목 처리(쉼표로 구분)
            items = [item.strip() for item in row[field].split(",")]
            optional_tokens.extend([f"{token} {item}" for item in items])
        else:
            optional_tokens.append(f"{token} <empty>")


    ## 토큰 순서 섞기
    all_tokens = required_fields + optional_tokens
    random.shuffle(all_tokens)


    return "generate: " + " ".join(all_tokens) # prefix 추가

In [ ]:
## Input Text 생성
train_data["input_text"] = train_data.apply(generate_input_text, axis = 1)
val_data["input_text"] = val_data.apply(generate_input_text, axis = 1)

### **output text**

In [ ]:
import re

In [ ]:
### Output Text 전처리 함수

def preprocess_output_text(story):
    if pd.isna(story):
        return "<empty>"

    story = re.sub(r'[^\w\s가-힣.,?!]', '', story) # 특수 문자 제거(문장 부호 유지)

    story = re.sub(r'\s+', ' ', story).strip() # 불필요한 공백 제거

    return story

In [ ]:
## Output Text 전처리

train_data["output_text"] = train_data["srcText"].apply(preprocess_output_text)
val_data["output_text"] = val_data["srcText"].apply(preprocess_output_text)

**데이터 확인**  

In [ ]:
train_data[["id", "input_text", "output_text"]].head()

,id,input_text,output_text
0,03_05T_03S_9791157023899_42712,generate: <i_action> 입에 한 손을 대고 놀란다 <character...,"플로렌스, 왜 리처드와 결혼을 하지 않겠다는 거니? 어머니가 물었어요. 언니 파세노..."
1,03_05T_03S_9791157023998_39246,generate: <outcomeResolution> <empty> <action>...,"그래, 일본으로 가자. 호랑이 굴로 들어가 보는 거야. 일본에서 벗어나려면 일본을 ..."
2,03_05T_03S_9791157023813_60463,generate: <prediction> <empty> <classification...,백 번을 죽고 또 죽어도 고려 임금을 향한 신의를 저버리지 않겠다는 뜻이었어요. 정...
3,03_05T_03S_9791157023981_34117,generate: <character> 바나나 <setting> 케임브리지 대학 <...,밀림을 떠나 케임브리지 대학에서 공부하고 왔는데도 침팬지들은 제인을 잊지 않았던 거...
4,03_05T_03S_9791157024247_33953,generate: <i_action> 앉아 있다 <causalRelationship...,"그런데 그날 저녁, 엄마가 뜻밖의 이야기를 하셨어요. 승찬아, 이번 주 일요일은 할..."


In [ ]:
val_data[["id", "input_text", "output_text"]].head()

,id,input_text,output_text
0,03_02T_03S_9788998212643_54879,generate: <action> 지키는 <character> 손잡이 <outcom...,출발합니다! 손잡이 꼭 잡으세요. 삐이익. 다음 정류장에 내릴 손님이 벨을 눌렀어요...
1,03_02T_03S_9788967760144_35692,generate: <i_action> 바다에 떠 있다 <character> 파도 <...,두 번째로 간 곳은 바다였다. 이번 모험은 고기잡이배에서 생활하는 것이다. 마음의 ...
2,03_02T_03S_9788998212636_5162,generate: <causalRelationship> <empty> <action...,제주도에 여행 온 친구들은 힝힝 씨가 들려주는 제주도 이야기를 아주 좋아해요. 제주...
3,03_02T_03S_9791128211331_32163,generate: <name> 난쟁이 <i_action> 침대에 누워있다 <feel...,쿠르쿠르는 에코스 덕분에 아주 오랜만에 단잠을 잘 수 있었어. 하지만 단 하루뿐이었...
4,03_02T_03S_9791128211195_59242,generate: <character> 적혈구 <setting> 핏속 <classi...,이 원반은 사람의 핏속에 있는 적혈구랍니다. 적혈구는 사람의 몸속 구석구석으로 산소...


# **2. Modeling**

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import json

## **2-1. model 설정**
- pre-trained `tokenizer`와 `model` 불러오기
- 한국어 말뭉치로 훈련된 koT5 활용
  - 현재는 **base** 활용
  - small version도 활용 가능

  ```python
  from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

  tokenizer = AutoTokenizer.from_pretrained("wisenut-nlp-team/KoT5-small")
  model = AutoModelForSeq2SeqLM.from_pretrained("wisenut-nlp-team/KoT5-small")
  ```

### **model file 구성**
1. **`config.json`**:
   - 모델 설정 파일
    - hidden size, layer 개수, attention heads 등
   - 모델 아키텍처와 관련된 정보를 포함

2. **`generation_config.json`**:
   - 텍스트 생성 시의 기본 설정값
    - `max_length`, `num_beams`, `repetition_penalty` 등

3. **`model.safetensors`**:
   - 모델의 가중치(weight)를 저장한 파일
    - `safetensors`는 더 안전하고 빠른 저장 형식을 사용

4. **`special_tokens_map.json`**:
   - 추가된 `special token`들의 정보를 명시적으로 저장
5. **`tokenizer_config.json`**:
   - `tokenizer`의 일반 설정
    - 사용된 vocabulary 크기, 특수 토큰 정보, tokenizer 클래스 이름 등을 저장
6. **`tokenizer.json`**:
   - `tokenizer`의 전체 단어 사전(vocabulary)과 매핑을 포함
   - 이 파일에 추가된 `special token`들이 반영되어, 새로운 토큰이 vocabulary에 추가된 것을 확인 가능

In [ ]:
## 모델 및 Tokenizer 로드
tokenizer = AutoTokenizer.from_pretrained("wisenut-nlp-team/KoT5-base", revision = "paraphrase")
model = AutoModelForSeq2SeqLM.from_pretrained("wisenut-nlp-team/KoT5-base", revision = "paraphrase")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.38k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.88M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.19G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

## **2-2. Tokenizer 설정**
- special token 추가 및 model, tokenizer 확장

In [ ]:
## 특수 token 정의
special_tokens = [
    "<caption>", "<name>", "<i_action>", "<classification>", "<character>",
    "<setting>", "<action>", "<feeling>", "<causalRelationship>",
    "<outcomeResolution>", "<prediction>", "<empty>"
]

In [ ]:
## 특수 token 확장
# 1. 기존 특수 토큰 가져오기
existing_special_tokens = tokenizer.special_tokens_map.get("additional_special_tokens", [])

# 2. 특수 토큰 병합 (중복 제거)
merged_special_tokens = list(set(existing_special_tokens + special_tokens))

# 3. 토크나이저에 특수 토큰 추가
tokenizer.add_special_tokens({'additional_special_tokens': merged_special_tokens})

# 4. 모델 Embedding 크기 재조정
model.resize_token_embeddings(len(tokenizer))

# 5. 모델과 토크나이저 저장
tokenizer.save_pretrained(model_path)
model.save_pretrained(model_path)

In [ ]:
print("Special tokens added:", tokenizer.special_tokens_map)
print("Vocabulary size:", len(tokenizer))

special_token_ids = tokenizer.convert_tokens_to_ids(special_tokens)
print("Special Token IDs:", sorted(special_token_ids))

Special tokens added: {'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<empty>', '<extra_id_74>', '<extra_id_94>', '<extra_id_0>', '<extra_id_67>', '<extra_id_77>', '<extra_id_16>', '<extra_id_15>', '<extra_id_20>', '<extra_id_53>', '<extra_id_88>', '<extra_id_79>', '<extra_id_49>', '<extra_id_7>', '<extra_id_97>', '<extra_id_99>', '<extra_id_2>', '<extra_id_69>', '<extra_id_28>', '<extra_id_14>', '<extra_id_30>', '<extra_id_98>', '<extra_id_1>', '<extra_id_45>', '<extra_id_93>', '<feeling>', '<extra_id_89>', '<extra_id_59>', '<extra_id_55>', '<extra_id_23>', '<character>', '<extra_id_90>', '<extra_id_84>', '<extra_id_39>', '<extra_id_27>', '<extra_id_12>', '<extra_id_29>', '<extra_id_41>', '<extra_id_92>', '<classification>', '<extra_id_58>', '<action>', '<extra_id_60>', '<extra_id_24>', '<extra_id_33>', '<extra_id_78>', '<outcomeResolution>', '<extra_id_8>', '<extra_id_76>', '<extra_id_52>', '<setting>', '<extra_id_51>', '<extra_id_75>'

## **2-3. Dataset 및 DataLoader 구성**
- model을 학습시키기 위한 데이터셋 구축 및 토큰화

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import DataCollatorForSeq2Seq

In [ ]:
### masking 함수 정의

def special_token_masking(input_ids, attention_mask, special_token_ids):
    mask_indices = (input_ids.unsqueeze(-1) == torch.tensor(special_token_ids)).any(dim=-1)
    attention_mask[mask_indices] = 0

    return attention_mask

In [ ]:
### Dataset 클래스 정의

class StoryDataset(Dataset):
    def __init__(self, dataframe, tokenizer, special_token_ids, max_length=128):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.special_token_ids = special_token_ids

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]

        input_encoding = self.tokenizer(
            row["input_text"],
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        target_encoding = self.tokenizer(
            row["output_text"],
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        # Special Token 마스킹 적용
        input_ids = input_encoding["input_ids"].squeeze(0)
        attention_mask = input_encoding["attention_mask"].squeeze(0)
        attention_mask = special_token_masking(input_ids, attention_mask, self.special_token_ids)

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": target_encoding["input_ids"].squeeze(0),
        }

In [ ]:
## Dataset 생성

train_dataset = StoryDataset(train_data, tokenizer, special_token_ids)
val_dataset = StoryDataset(val_data, tokenizer, special_token_ids)

In [ ]:
## DataLoader 생성

# Dynamic padding 적용
data_collator = DataCollatorForSeq2Seq(tokenizer, model = model)

train_dataloader = DataLoader(train_dataset, batch_size = 4, shuffle = True, collate_fn = data_collator)
val_dataloader = DataLoader(val_dataset, batch_size = 4, collate_fn = data_collator)

## **2-4. 샘플 확인**
- special token에 대한 처리가 제대로 진행되고 있는지 확인

In [ ]:
# Tokenizer가 특수 토큰을 정확히 인식하는지 확인
for token in special_tokens:
    token_id = tokenizer.convert_tokens_to_ids(token)
    print(f"Token: {token}, Token ID: {token_id}, Decoded: {tokenizer.decode([token_id])}")

Token: <caption>, Token ID: 32108, Decoded: <caption>
Token: <name>, Token ID: 32109, Decoded: <name>
Token: <i_action>, Token ID: 32107, Decoded: <i_action>
Token: <classification>, Token ID: 32103, Decoded: <classification>
Token: <character>, Token ID: 32102, Decoded: <character>
Token: <setting>, Token ID: 32106, Decoded: <setting>
Token: <action>, Token ID: 32104, Decoded: <action>
Token: <feeling>, Token ID: 32101, Decoded: <feeling>
Token: <causalRelationship>, Token ID: 32110, Decoded: <causalRelationship>
Token: <outcomeResolution>, Token ID: 32105, Decoded: <outcomeResolution>
Token: <prediction>, Token ID: 32111, Decoded: <prediction>
Token: <empty>, Token ID: 32100, Decoded: <empty>


In [ ]:
## Tokenization 상세 확인

# 샘플 배치 확인
sample_batch = next(iter(train_dataloader))
input_ids = sample_batch['input_ids']
attention_mask = sample_batch['attention_mask']

for i in range(4):  # 샘플 4개 확인
    text = sample_batch['input_ids'][i]
    tokens = tokenizer.convert_ids_to_tokens(text)
    mask = sample_batch['attention_mask'][i].tolist()

    print(f"Decoded Text {i+1}: {tokenizer.decode(text, skip_special_tokens=False)}")
    print(f"Tokens: {tokens}")
    print(f"Attention Mask: {mask}")
    print("=" * 50)

Decoded Text 1: generate: <setting> 며칠 동안 <outcomeResolution> <empty> <feeling> 자신감이 생겼어요 <action> 찾아서 <action> 떨어져라 <caption> 표지판을 들고 있는 소녀의 그림 <action> 연습했어요 <causalRelationship> <empty> <character> 주문 <i_action> 손에 들려 있다 <character> 지폐 <prediction> <empty> <classification> 의사소통 <feeling> 마음에 안 드네 <name> 지폐 <character> 눈물방울</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
Tokens: ['▁g', 'en', 'er', 'ate', ':', '▁', '<setting>', '▁며칠', '▁동안', '▁', '<outcomeResolution>', '▁', '<empty>', '▁', '<feeling>', '▁자신감이', '▁생겼', '어요', '▁', '<action>', '▁찾아서', '▁', '<action>', '▁떨어져', '라', '▁', '<caption>', '▁표지', '판을', '▁들고', '▁있는', '▁소녀의', '▁그림', '▁', '<action>', '▁연습', '했어요', '▁', '<causalRelationship>', '▁', '<empty>', '▁', '<character>', '▁주문', '▁', '<i_action>', '

/usr/local/lib/python3.11/dist-packages/transformers/data/data_collator.py:657: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


In [ ]:
## 특수 토큰 전후 확인

for i in range(4):
    tokens = tokenizer.convert_ids_to_tokens(input_ids[i])
    attention = attention_mask[i].tolist()

    for idx, (token, mask) in enumerate(zip(tokens, attention)):
        if token in special_tokens or token == "<empty>":
            print(f"Token: {token}, Mask: {mask}, Prev: {attention[idx-1] if idx > 0 else 'N/A'}, Next: {attention[idx+1] if idx < len(tokens)-1 else 'N/A'}")
    print("="*50)

Token: <setting>, Mask: 0, Prev: 1, Next: 1
Token: <outcomeResolution>, Mask: 0, Prev: 1, Next: 1
Token: <empty>, Mask: 0, Prev: 1, Next: 1
Token: <feeling>, Mask: 0, Prev: 1, Next: 1
Token: <action>, Mask: 0, Prev: 1, Next: 1
Token: <action>, Mask: 0, Prev: 1, Next: 1
Token: <caption>, Mask: 0, Prev: 1, Next: 1
Token: <action>, Mask: 0, Prev: 1, Next: 1
Token: <causalRelationship>, Mask: 0, Prev: 1, Next: 1
Token: <empty>, Mask: 0, Prev: 1, Next: 1
Token: <character>, Mask: 0, Prev: 1, Next: 1
Token: <i_action>, Mask: 0, Prev: 1, Next: 1
Token: <character>, Mask: 0, Prev: 1, Next: 1
Token: <prediction>, Mask: 0, Prev: 1, Next: 1
Token: <empty>, Mask: 0, Prev: 1, Next: 1
Token: <classification>, Mask: 0, Prev: 1, Next: 1
Token: <feeling>, Mask: 0, Prev: 1, Next: 1
Token: <name>, Mask: 0, Prev: 1, Next: 1
Token: <character>, Mask: 0, Prev: 1, Next: 1
Token: <i_action>, Mask: 0, Prev: 1, Next: 1
Token: <action>, Mask: 0, Prev: 1, Next: 1
Token: <character>, Mask: 0, Prev: 1, Next: 1
Toke

# **3. Training**

In [ ]:
!pip install evaluate

In [ ]:
!pip install rouge_score

## **3-1. Configs**

In [ ]:
from transformers import get_scheduler
from tqdm.auto import tqdm
import torch
from torch.optim import AdamW
import evaluate

In [ ]:
## 장치 설정 및 모델 확인

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32112, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32112, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [ ]:
## 수정 필요

LEARNING_RATE = 3e-5
EPOCHS = 1
WARMUP_STEPS = 500
PATIENCE = 3  # Early Stopping

In [ ]:
## Optimizer 및 Scheduler 설정
optimizer = AdamW(model.parameters(), lr = LEARNING_RATE)
num_training_steps = EPOCHS * len(train_dataloader)

lr_scheduler = get_scheduler(
    "linear",
    optimizer = optimizer,
    num_warmup_steps = WARMUP_STEPS,
    num_training_steps = num_training_steps
)

In [ ]:
## 평가 지표 설정
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

In [ ]:
## Loss Function
PAD_TOKEN_ID = tokenizer.pad_token_id
loss_fn = torch.nn.CrossEntropyLoss(ignore_index = PAD_TOKEN_ID)

## **3-2. Run!!**

In [ ]:
best_val_loss = float("inf")
early_stop_counter = 0

In [ ]:
# torch cache 비우기

import gc

gc.collect()
torch.cuda.empty_cache()

In [ ]:
### Training/Validation Loop

for epoch in range(EPOCHS):
    print(f"\n=== Epoch {epoch + 1}/{EPOCHS} ===")

    ## Training
    model.train()
    train_loss = 0.0

    for batch in tqdm(train_dataloader, desc="Training"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        # forward pass(logit 반환)
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

        loss = outputs.loss
        train_loss += loss.item()

        # back propagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

    avg_train_loss = train_loss / len(train_dataloader)
    print(f"Train Loss: {avg_train_loss:.4f}")


    ## Validation
    model.eval()
    val_loss = 0.0
    preds, actuals = [], []

    with torch.no_grad():
        for batch in tqdm(val_dataloader, desc = "Validation"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            # forward pass(logit 반환)
            outputs = model(
                input_ids = input_ids,
                attention_mask = attention_mask,
                labels = labels
            )

            loss = outputs.loss
            val_loss += loss.item()

            # Text Generation(실제 문장 생성)
            generated_ids = model.generate(
                input_ids = input_ids,
                attention_mask = attention_mask,
                max_length = 128,
                num_beams = 3,
                length_penalty = 1.0,
                repetition_penalty = 2.5,
                no_repeat_ngram_size = 2,
                early_stopping = True,
            )

            batch_preds = tokenizer.batch_decode(generated_ids, skip_special_tokens = True, clean_up_tokenization_spaces = True)
            batch_actuals = tokenizer.batch_decode(labels, skip_special_tokens = True, clean_up_tokenization_spaces = True)

            preds.extend(batch_preds)
            actuals.extend(batch_actuals)


    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Validation Loss: {avg_val_loss:.4f}")


    ## Sample Output
    print("\n=== Sample Predictions and References ===")
    for i in range(min(3, len(preds))):
        print(f"Prediction {i + 1}: {preds[i]}")
        print(f"Reference {i + 1}: {actuals[i]}")


    ## Evaluation
    try:
        bleu_result = bleu.compute(predictions = preds, references = [[r] for r in actuals])
        rouge_result = rouge.compute(predictions = preds, references = actuals)
        print(f"BLEU Score: {bleu_result['bleu']:.4f}")
        print(f"ROUGE Scores: {rouge_result}")
    except Exception as e:
        print(f"Error during BLEU or ROUGE computation: {e}")


    ## Early Stopping & model checkpoint 저장
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        early_stop_counter = 0
        model.save_pretrained(f"{model_path}/best_model")
        tokenizer.save_pretrained(f"{model_path}/best_model")
        print("✅ Best model saved.")
    else:
        early_stop_counter += 1
        if early_stop_counter >= PATIENCE:
            print("⏹️ Early stopping triggered.")
            break

    # Clear Cache After Each Epoch
    gc.collect()
    torch.cuda.empty_cache()


=== Epoch 1/1 ===


Training:   0%|          | 0/10001 [00:00<?, ?it/s]

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Train Loss: 1.5094


Validation:   0%|          | 0/1250 [00:00<?, ?it/s]

Validation Loss: 1.1795

=== Sample Predictions and References ===
Prediction 1: 정류장에서 신호등이 멈췄다 가자, 차돌이 멈춰 섰어요. 신호등에서 내릴 때는 반드시 손잡이 버튼을 눌러야 해요.
Reference 1: 출발합니다! 손잡이 꼭 잡으세요. 삐이익. 다음 정류장에 내릴 손님이 벨을 눌렀어요. 차돌아, 빨간불이 들어왔구나. 잠깐 멈췄다 가자. 신호등은 꼭 지켜야 해요. 안전 운전은 생명을 지키는 일이거든요.
Prediction 2: 나는 하루 종일 고기잡이배를 타고 먼 바다로 출발했다. 그물을 끌어 올려야 해. 물고기를 태운 배는 파도에 휩쓸리지 않고 안전하게 헤엄을 칠 수 있어.
Reference 2: 두 번째로 간 곳은 바다였다. 이번 모험은 고기잡이배에서 생활하는 것이다. 마음의 준비를 할 새도 없이 나를 태운 고기잡이배는 멀고 먼 바다로 출발했다. 나는 먼저 물고기를 잡은 그물부터 손질해야 했다. 밥도 제대로 먹지 못하고 하루 종일 그물을 끌어 올려야 했던 날도 많았다. 또 물고기는 한 마리도 구경하지 못한 채 집채만 한 파도와 싸워야 했던 날도 있었다.
Prediction 3: 해녀가 들려주는 이야기를 좋아해요. 힝힝 씨는 제주도의 섬에서 살아요.
Reference 3: 제주도에 여행 온 친구들은 힝힝 씨가 들려주는 제주도 이야기를 아주 좋아해요. 제주도에는 누워 있는 소처럼 생긴 섬이 있어. 바닷속에서 물고기와 조개를 잡는 해녀도 있지.
BLEU Score: 0.0682
ROUGE Scores: {'rouge1': 0.02415173382173383, 'rouge2': 0.007353809523809524, 'rougeL': 0.023716868131868143, 'rougeLsum': 0.023554328449328456}
✅ Best model saved.
